In [1]:
import pandas as pd
import numpy as np
import pickle
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import nltk
nltk.download('wordnet')
from text_cleaning import lemmatize_and_stem, preprocess
import time
import urllib.request
from collections import Counter
from selenium.webdriver import Chrome
from scrape import get_wiki_description, replace_df_text

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/kaylischulz/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
countries_df = pickle.load(open('countries.pkl', 'rb'))
countries_df.head()

,country,country_summary
0,Austria,"Small, landlocked Austria offers alpine scener..."
1,Belgium,Belgium falls through the cracks. Wedged betwe...
2,Bosnia-Herzegovina,Apart from the tragic way it separated from Yu...
3,Bulgaria,"Endearing, surprising Bulgaria is a rewarding ..."
4,Croatia,With thousands of miles of seafront and more t...


In [3]:
cities_df = pickle.load(open('cities.pkl', 'rb'))
cities_df.head()

,city,city_summary,country
0,Danube Valley,The Danube is at its romantic best just west o...,Austria
1,Hallstatt,Lovable Hallstatt is a tiny town bullied onto ...,Austria
2,Salzburg,"Thanks to its charmingly preserved old town, s...",Austria
3,Tirol,Mountainous Tirol — in Austria's western panha...,Austria
4,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria


In [4]:
wiki_city_df = pickle.load(open('wiki_data.pkl', 'rb'))

In [5]:
wiki_city_df.head()

,_id,city,text
0,5d1195dc0d668bb1db943136,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
1,5d11970b6e7463927710e427,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
2,5d11974d6e7463927710e428,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
3,5d1197626e7463927710e429,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
4,5d1197936e7463927710e42a,Hallstatt,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...


In [6]:
wiki_city_df.drop('_id', axis=1, inplace=True)
wiki_city_df.head()

,city,text
0,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
1,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
2,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
3,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
4,Hallstatt,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...


In [7]:
wiki_city_df.drop_duplicates(inplace=True)

In [8]:
wiki_city_df.head()

,city,text
0,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
2,Danube Valley,\nThe Danube (/ˈdæn.juːb/ DAN-yoob; known by v...
4,Hallstatt,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...
5,Salzburg,Salzburg (German: [ˈzaltsbʊɐ̯k] (listen);[note...
6,Tirol,"\nTyrol (/tɪˈroʊl, taɪ-, ˈtaɪroʊl/;[1] histori..."


In [10]:
for row in wiki_city_df['text']:
    print(row)
    time.sleep(2)


Vienna (/viˈɛnə/ (listen);[11][12] German: Wien [viːn] (listen)) is the federal capital, largest city and one of nine states of Austria. Vienna is Austria's primate city, with a population of about 1.9 million[3] (2.6 million within the metropolitan area,[6] nearly one third of the country's population), and its cultural, economic, and political centre. It is the 7th-largest city by population within city limits in the European Union. Until the beginning of the 20th century, it was the largest German-speaking city in the world, and before the splitting of the Austro-Hungarian Empire in World War I, the city had 2 million inhabitants.[13] Today, it has the second largest number of German speakers after Berlin.[14][15] Vienna is host to many major international organizations, including the United Nations and OPEC. The city is located in the eastern part of Austria and is close to the borders of the Czech Republic, Slovakia, and Hungary. These regions work together in a European Centrope


The Danube (/ˈdæn.juːb/ DAN-yoob; known by various names in other languages) is Europe's second longest river, after the Volga. It is located in Central and Eastern Europe.
The Danube was once a long-standing frontier of the Roman Empire, and today flows through 10 countries, more than any other river in the world. Originating in Germany, the Danube flows southeast for 2,850 km (1,770 mi), passing through or bordering Austria, Slovakia, Hungary, Croatia, Serbia, Romania, Bulgaria, Moldova and Ukraine before draining into the Black Sea. Its drainage basin extends into nine more countries.
The Danube river basin is home to fish species such as pike, zander, huchen, Wels catfish, burbot and tench. It is also home to a large diversity of carp and sturgeon, as well as salmon and trout. A few species of euryhaline fish, such as European seabass, mullet, and eel inhabit the Danube Delta and the lower portion of the river.
Since ancient times, the Danube has become a traditional trade route i

Hallstatt (German: [ˈhalʃtat]; Central Bavarian: Hoistod) is a small town in the district of Gmunden, in the Austrian state of Upper Austria. Situated between the southwestern shore of Hallstätter See and the steep slopes of the Dachstein massif, the town lies in Salzkammergut region, on the national road linking Salzburg and Graz.
Hallstatt is known for its production of salt, dating back to prehistoric times, and gave its name to the Hallstatt culture, the archaeological culture linked to Proto-Celtic and early Celtic people of the Early Iron Age in Europe, c.800–450 BC.
Hallstatt is at the core of the "Hallstatt-Dachstein/Salzkammergut Cultural Landscape" declared as one of the World Heritage Sites in Austria by UNESCO in 1997.
In 1846, Johann Georg Ramsauer (1795–1874) discovered a large prehistoric cemetery at the Salzberg mines near Hallstatt (47.561°N 13.642°E), which he excavated during the second half of the 19th century. Eventually the excavation would yield 1,045 burials, al


Tyrol (/tɪˈroʊl, taɪ-, ˈtaɪroʊl/;[1] historically the Tyrole;[2][3][4] German: Tirol [tiˈʁoːl] (listen); Italian: Tirolo) is a historical region in the Alps; in Northern Italy and western Austria. The area was to Habsburg times the former County of Tyrol. The Treaty of Saint-Germain-en-Laye in 1919 divided the area into two parts:
With the founding of the European region Tyrol-South Tyrol-Trentino the area has its own legal entity since 2011 in the form of a European Grouping for Territorial Cooperation.
According to Egon Kühebacher [de], the name Tyrol derives from a root word meaning terrain (i.e. area, ground or soil; compare Latin: terra and Old Irish: tir); first from the village of Tirol, and its castle; from which the County of Tyrol grew.[5] According to Karl Finsterwalder, the name Tyrol derives from Teriolis [de], a late-Roman fort and travellers' hostel in Zirl, Tyrol.[6] There seems to be no scholarly consensus.
Tyrol has an area of 26,673 km2. The region consists of the S


Antwerp (/ˈæntwɜːrp/ (listen); Dutch: Antwerpen [ˈɑntʋɛrpə(n)] (listen); French: Anvers [ɑ̃vɛʁs]) is a city in Belgium, and is the capital of Antwerp province in Flanders. With a population of 520,504,[2] it is the most populous city proper in Belgium, and with a metropolitan area housing around 1,200,000 people, it's the second largest metropolitan region after Brussels in Belgium.[3][4]
Antwerp is on the River Scheldt, linked to the North Sea by the river's Westerschelde estuary. It is about 40 kilometres (25 mi) north of Brussels, and about 15 kilometres (9 mi) south of the Dutch border. The Port of Antwerp is one of the biggest in the world, ranking second in Europe[5][6] and within the top 20 globally.[7] The city is also known for its diamond industry and trade.
Both economically and culturally, Antwerp is and has long been an important city in the Low Countries, especially before and during the Spanish Fury (1576) and throughout and after the subsequent Dutch Revolt. Antwerp wa

...


The Hague (/heɪɡ/; Dutch: Den Haag [dɛn ˈɦaːx] (listen) or 's-⁠Gravenhage [ˌsxraːvə(n)ˈɦaːɣə] (listen)) is a city on the western coast of the Netherlands and the capital of the province of South Holland. It is also the seat of government of the Netherlands.
With a metropolitan population of more than 1 million, it is the third-largest city in the Netherlands, after Amsterdam and Rotterdam. The Rotterdam–The Hague metropolitan area, with a population of approximately 2.7 million, is the 13th-largest in the European Union and the most populous in the country. Located in the west of the Netherlands, The Hague is in the centre of the Haaglanden conurbation and lies at the southwest corner of the larger Randstad conurbation.
The Hague is the seat of the Cabinet, the States General, the Supreme Court, and the Council of State of the Netherlands, but the city is not the constitutional capital of the Netherlands, which is Amsterdam.[8] King Willem-Alexander lives in Huis ten Bosch[9][a] and w

Bergen (Norwegian pronunciation: [ˈbærɡn̩] (listen)), historically Bjørgvin, is a city and municipality in Hordaland on the west coast of Norway. At the end of the first quarter of 2018, the municipality's population was 280,216,[1] and the Bergen metropolitan region has about 420,000 inhabitants. Bergen is the second-largest city in Norway. The municipality covers 465 square kilometres (180 sq mi) and is on the peninsula of Bergenshalvøyen. The city centre and northern neighbourhoods are on Byfjorden, 'the city fjord', and the city is surrounded by mountains; Bergen is known as the 'city of seven mountains'. Many of the extra-municipal suburbs are on islands. Bergen is the administrative centre of Hordaland, and consists of eight boroughs: Arna, Bergenhus, Fana, Fyllingsdalen, Laksevåg, Ytrebygda, Årstad, and Åsane.
Trading in Bergen may have started as early as the 1020s. According to tradition, the city was founded in 1070 by king Olav Kyrre and was named Bjørgvin, 'the green meadow

This list of Norwegian fjords shows many of the fjords in Norway. In total, there are about 1,190 fjords in Norway and the Svalbard islands. The sortable list includes the lengths and locations of those fjords.



Oslo (/ˈɒzloʊ/ OZ-loh, also US: /ˈɒsloʊ/ OSS-loh,[9][10] Norwegian: [²ʊʂlʊ] (listen), rarely [²ʊslʊ, ˈʊʂlʊ]) is the capital and most populous city of Norway. It constitutes both a county and a municipality. Founded in the year 1040 as Ánslo, and established as a kaupstad or trading place in 1048 by Harald Hardrada, the city was elevated to a bishopric in 1070 and a capital under Haakon V of Norway around 1300. Personal unions with Denmark from 1397 to 1523 and again from 1536 to 1814 reduced its influence. After being destroyed by a fire in 1624, during the reign of King Christian IV, a new city was built closer to Akershus Fortress and named Christiania in the king's honour. It was established as a municipality (formannskapsdistrikt) on 1 January 1838. The city functioned as



The Auschwitz concentration camp (Konzentrationslager Auschwitz) was a complex of over 40 concentration and extermination camps built and operated by Nazi Germany in occupied Poland during World War II and the Holocaust. It consisted of Auschwitz I, the main camp (Stammlager) and administrative headquarters in Oświęcim; Auschwitz II–Birkenau, a combined concentration and extermination camp three kilometers away in Brzezinka; Auschwitz III–Monowitz, a labor camp created to staff an IG Farben synthetic-rubber factory; and dozens of other subcamps.[2]
After Germany invaded Poland in September 1939, sparking World War II, the Germans converted Auschwitz I, a former army barracks, to hold Polish political prisoners.[3] The first prisoners, German criminals brought to the camp as functionaries, arrived in May 1940,[4] and the first gassing of prisoners took place in block 11 of Auschwitz I in September 1941. Auschwitz II–Birkenau went on to become a major site of the Nazis' Final Solution 

KeyboardInterrupt: 

Since the data set is small enough and it is possible that wikipedia sent to a different link than intended (due to ambiguity), I will do a visual check that the summary is the summary I want.

Cities I need to recollect:
* Danube Valley, Austria (references the river, I want to get the valley (city such as Melk))
* Split, Croatia
* Bath, England
* Durham, Englang
* Glastonbury & Wells, England (may need to get for each city?)
* Stonehenge & Avebury, England (may need to refer to each city/landmark)
* Stratford, England
* Warwick & Coventry, England
* Windsor, England
* D-Day Beaches, France
* Reims & Verdun, France (may need to get for each city)
* Rhine Valley, Germany
* Hydra, Greece
* Olympia, Greece
* Connemara & County Mayo, Ireland
* Country Clare & the Burren, Ireland
* Kenmare & the Ring of Kerry, Ireland
* Kilkenny & the Rock of Cashel, Ireland
* Kinsale & Cobh, Ireland
* Portrush & the Antrim Coast, Ireland
* Waterford & County Wexford, Ireland
* Italian Lakes, Italy
* Pompeii & Herculaneum, Italy
* Tuscan Hill Towns, Italy
* Edam, Netherlands
* Norwegian Fjords, Norway
* Nazaré, Portugal
* Óbidos, Portugal
* Oban, Mull & Iona, Scotland
* Córdoba, Spain
* Toledo, Spain
* White Hill Towns, Spain
* Lake Geneva & French Switzerland, Switzerland


In [14]:
browser = Chrome()

In [ ]:
replace_df_text(browser, ('Danube Valley', 'Melk'), wiki_city_df)

In [11]:
wiki_city_df.head()

,city,text
0,Vienna,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
2,Danube Valley,Melk (older spelling: Mölk; Central Bavarian: ...
4,Hallstatt,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...
5,Salzburg,Salzburg (German: [ˈzaltsbʊɐ̯k] (listen);[note...
6,Tirol,"\nTyrol (/tɪˈroʊl, taɪ-, ˈtaɪroʊl/;[1] histori..."


In [12]:
city_wiki = [('Danube Valley', 'Melk'),
             ('Split', 'Split,_Croatia'), ('Bath', 'Bath,_Somerset'), ('Durham', 'Durham,_England'), 
             ('Glastonbury & Wells', 'Glastonbury'), ('Stonehenge & Avebury', 'Avebury'),
             ('Stratford', 'Stratford-upon-Avon'), ('Warwick & Coventry', 'Coventry'),
             ('Windsor', 'Windsor,_Berkshire'), ('D-Day Beaches', 'Normandy_landings'),
             ('Reims & Verdun', 'Reims'), ('Rhine Valley', 'Mörsbach'), ('Hydra', 'Hydra_(island)'),
             ('Olympia', 'Olympia,_Greece'), ('Connemara & County Mayo', 'Connemara'),
             ('Country Clare & the Burren', 'The_Burren'), ('Kenmare & the Ring of Kerry', 'Kenmare'),
             ('Kilkenny & the Rock of Cashel', 'Kilkenny'), ('Kinsale & Cobh', 'Cobh'),
             ('Portrush & the Antrim Coast', 'Portrush'), ('Waterford & County Wexford', 'Waterford'),
             ('Italian Lakes', 'Varenna'), ('Pompeii & Herculaneum', 'Pompeii'),
             ('Tuscan Hill Towns', 'Montepulciano'), ('Edam', 'Edam,_Netherlands'),
             ('Norwegian Fjords', 'Sognefjord'), ('Nazaré', 'Nazaré,_Portugal'),
             ('Óbidos', 'Óbidos,_Portugal'), ('Oban, Mull & Iona', 'Iona'), ('Córdoba', 'Córdoba,_Spain'),
             ('Toledo', 'Toledo,_Spain'), ('White Hill Towns', 'Ronda'),
             ('Lake Geneva & French Switzerland', 'Lake_Geneva'),
            ]

In [15]:
for city in city_wiki:
    replace_df_text(browser, city, wiki_city_df)
    time.sleep(np.random.randint(10,30))
    print(f'Replaced {city[0]}')

Replaced Danube Valley
Replaced Split
Replaced Bath
Replaced Durham
Replaced Glastonbury & Wells
Replaced Stonehenge & Avebury
Replaced Stratford
Replaced Warwick & Coventry
Replaced Windsor
Replaced D-Day Beaches
Replaced Reims & Verdun
Replaced Rhine Valley
Replaced Hydra
Replaced Olympia
Replaced Connemara & County Mayo
Replaced Country Clare & the Burren
Replaced Kenmare & the Ring of Kerry
Replaced Kilkenny & the Rock of Cashel
Replaced Kinsale & Cobh
Replaced Portrush & the Antrim Coast
Replaced Waterford & County Wexford
Replaced Italian Lakes
Replaced Pompeii & Herculaneum
Replaced Tuscan Hill Towns
Replaced Edam
Replaced Norwegian Fjords
Replaced Nazaré
Replaced Óbidos
Replaced Oban, Mull & Iona
Replaced Córdoba
Replaced Toledo
Replaced White Hill Towns
Replaced Lake Geneva & French Switzerland


In [21]:
combined_descriptions_df = pd.merge(cities_df, wiki_city_df, how='left', on='city')
combined_descriptions_df.head(10)

,city,city_summary,country,text
0,Danube Valley,The Danube is at its romantic best just west o...,Austria,Melk (older spelling: Mölk; Central Bavarian: ...
1,Hallstatt,Lovable Hallstatt is a tiny town bullied onto ...,Austria,Hallstatt (German: [ˈhalʃtat]; Central Bavaria...
2,Salzburg,"Thanks to its charmingly preserved old town, s...",Austria,Salzburg (German: [ˈzaltsbʊɐ̯k] (listen);[note...
3,Tirol,Mountainous Tirol — in Austria's western panha...,Austria,"\nTyrol (/tɪˈroʊl, taɪ-, ˈtaɪroʊl/;[1] histori..."
4,Vienna,"Vienna is the capital of Austria, the cradle o...",Austria,\nVienna (/viˈɛnə/ (listen);[11][12] German: W...
5,Antwerp,"Antwerp (Antwerpen in Dutch, Anvers in French)...",Belgium,\nAntwerp (/ˈæntwɜːrp/ (listen); Dutch: Antwer...
6,Bruges,"With pointy gilded architecture, stay-a-while ...",Belgium,"\nBruges (/bruːʒ/, French: [bʁyʒ]; Flemish: Br..."
7,Brussels,"Six hundred years ago, Brussels was just a nic...",Belgium,\nBrussels (French: Bruxelles [bʁysɛl] (listen...
8,Ghent,"Ghent doesn't ooze with cobbles and charm, lik...",Belgium,\nGhent (/ɡɛnt/; Flemish: Gent [ɣɛnt] (listen)...
9,Mostar,"Despite the scars of war, Mostar is still stun...",Bosnia-Herzegovina,Mostar is a city and the administrative center...


In [2]:
#pickle.dump(combined_descriptions_df, open('combined_cities.pkl', 'wb'))
combined_descriptions_df = pickle.load(open('combined_cities.pkl', 'rb'))

In [3]:
processed_countries = list(combined_descriptions_df['city_summary'].map(preprocess))
processed_countries.extend(list(combined_descriptions_df['text'].map(preprocess)))

In [4]:
len(processed_countries)

426

In [5]:
processed_countries[-1]

['welsh',
 'cymru',
 'region',
 'border',
 'bristol',
 'channel',
 'mid',
 'estim',
 'million',
 'quarter',
 'cardiff',
 'approxim',
 'swansea',
 'approxim',
 'newport',
 'region',
 'loos',
 'defin',
 'gener',
 'consid',
 'includ',
 'histor',
 'counti',
 'glamorgan',
 'monmouthshir',
 'extend',
 'westward',
 'includ',
 'carmarthenshir',
 'pembrokeshir',
 'extent',
 'swansea',
 'westward',
 'local',
 'peopl',
 'probabl',
 'recognis',
 'live',
 'brecon',
 'beacon',
 'nation',
 'park',
 'cover',
 'contain',
 'pen',
 'fan',
 'highest',
 'mountain',
 'cadair',
 'idri',
 'snowdonia',
 'statut',
 'rhuddlan',
 'law',
 'act',
 'crown',
 'form',
 'princip',
 'divid',
 'princip',
 'princip',
 'princip',
 'counti',
 'ceredigion',
 'carmarthenshir',
 'area',
 'previous',
 'welsh',
 'kingdom',
 'deheubarth',
 'legal',
 'respons',
 'lay',
 'hand',
 'justiciar',
 'base',
 'carmarthen',
 'part',
 'hand',
 'marcher',
 'lord',
 'law',
 'act',
 'creat',
 'court',
 'great',
 'session',
 'base',
 'legal',
 

In [19]:
dictionary_bow = gensim.corpora.Dictionary(processed_countries)

In [20]:
dictionary_bow.filter_extremes(no_below=10, no_above=0.35)

In [21]:
bow_corpus = [dictionary_bow.doc2bow(country) for country in processed_countries]

In [22]:
tfidf = gensim.models.TfidfModel(bow_corpus)
corpus_tfidf = tfidf[bow_corpus]

In [ ]:
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10,
                                       id2word=dictionary_bow, passes=20,
                                       workers=1, alpha=.5, )

In [ ]:
for ind, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(ind, topic))

In [ ]:
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary_bow, passes=20, workers=1)

In [ ]:
for ind, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(ind, topic))

In [14]:
combined_descriptions_df.iloc[8]['city_summary']

"Ghent doesn't ooze with cobbles and charm, like its rival Bruges — this is a living city, with an urban grittiness and a welcome splash of creative hipster funkiness. Explore its historic quarter, ogle the Van Eyck altarpiece in its massive cathedral, tour its impressive art and design museums, stroll its picturesque embankments, bask in its finely decorated historic gables, and prowl its newly revitalized Patershol restaurant quarter. Ghent is the kind of town that you visit for a few hours…and find yourself wishing you had a few days."

In [15]:
for ind, score in sorted(lda_model[bow_corpus[8]], key=lambda tup: -1*tup[1]):
    print('\nScore: {}\t \nTopic: {}'.format(score, lda_model.print_topic(ind, 10)))


Score: 0.403144508600235	 
Topic: 0.005*"intern" + 0.004*"univers" + 0.004*"art" + 0.004*"camp" + 0.004*"park" + 0.003*"plan" + 0.003*"music" + 0.003*"compani" + 0.003*"oper" + 0.003*"architectur"

Score: 0.18965762853622437	 
Topic: 0.009*"bridg" + 0.008*"council" + 0.007*"bosnia" + 0.006*"herzegovina" + 0.005*"al" + 0.005*"king" + 0.005*"shakespear" + 0.004*"theatr" + 0.004*"londonderri" + 0.004*"castl"

Score: 0.14095480740070343	 
Topic: 0.020*"lake" + 0.020*"valley" + 0.019*"forest" + 0.014*"mountain" + 0.007*"railway" + 0.006*"lie" + 0.006*"alp" + 0.006*"natur" + 0.005*"pass" + 0.005*"canton"

Score: 0.1271539330482483	 
Topic: 0.009*"univers" + 0.006*"palac" + 0.005*"art" + 0.005*"council" + 0.005*"park" + 0.005*"public" + 0.005*"school" + 0.004*"king" + 0.004*"squar" + 0.004*"bridg"

Score: 0.04123655706644058	 
Topic: 0.008*"art" + 0.007*"pope" + 0.007*"di" + 0.005*"piazza" + 0.005*"renaiss" + 0.004*"emperor" + 0.004*"palazzo" + 0.004*"empir" + 0.004*"del" + 0.004*"architectu

In [16]:
for ind, score in sorted(lda_model_tfidf[bow_corpus[8]], key=lambda tup: -1*tup[1]):
    print('\nScore: {}\t \nTopic: {}'.format(score, lda_model_tfidf.print_topic(ind, 10)))


Score: 0.9368582963943481	 
Topic: 0.002*"island" + 0.002*"univers" + 0.002*"castl" + 0.001*"mountain" + 0.001*"lake" + 0.001*"art" + 0.001*"port" + 0.001*"cathedr" + 0.001*"coast" + 0.001*"beach"

Score: 0.036455653607845306	 
Topic: 0.002*"sevil" + 0.002*"revit" + 0.001*"andalusian" + 0.001*"horizon" + 0.001*"wander" + 0.001*"scar" + 0.001*"puerta" + 0.001*"málaga" + 0.001*"sierra" + 0.001*"propel"


In [17]:
train_vecs = []
for i in range(len(bow_corpus)):
    top_topics = lda_model.get_document_topics(bow_corpus[i], minimum_probability=0.0)
    topic_vec = [top_topics[i][1] for i in range(10)]
    train_vecs.append(topic_vec)

In [18]:
train_vecs[210]

[0.011466329,
 0.110250436,
 0.011594969,
 0.012493741,
 0.012492148,
 0.013472825,
 0.013157714,
 0.018792445,
 0.5608825,
 0.2353969]